In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [2]:
train_df = pd.read_csv('./data/train.csv')
serving_df = pd.read_csv('./data/test.csv')

train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
def preprocess(df: pd.DataFrame): 
    df = df.copy()
    
    def find_non_chars(df: list): 
        import re
        # Regular expression pattern to find characters that are not A-Z
        pattern = re.compile('[^a-zA-Z\s]')

        # List to store non-alphabet characters
        non_alphabet_chars = []

        # Iterate over each name and find non-alphabet characters
        for name in df['Name']:
            non_alphabet_chars.extend(pattern.findall(name))

        return ''.join(list(set(non_alphabet_chars)))
    
    non_alphabet_chars = find_non_chars(df)
    
    def normalize_name(s: str): 
        return ' '.join([x.strip(non_alphabet_chars) for x in s.split(' ')])
    
    def ticket_number(x: str): 
        return x.split(' ')[-1]
    
    def ticket_item(x: str): 
        items = x.split(' ')
        if len(items) == 1: 
            return None
        return ''.join(items[0:-1])
    
    df['Name'] = df['Name'].apply(normalize_name)
    df['Ticket_number'] = df['Ticket'].apply(ticket_number)
    df['Ticket_item'] = df['Ticket'].apply(ticket_item)
    return df

preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)
preprocessed_train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,None
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,None


In [4]:
input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")

print(f"Input features: {input_features}")

Input features: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


## Convert Pandas dataset to TensorFlow Dataset

In [5]:
def tokenize_names(features, labels= None): 
    features['Name'] = tf.strings.split(features['Name'])
    return features, labels
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df, label='Survived').map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)

2024-01-12 15:38:54.690446: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-01-12 15:38:54.690464: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-01-12 15:38:54.690469: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-01-12 15:38:54.690497: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-12 15:38:54.690511: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Train model with default parameters

In [6]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, 
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features], 
    exclude_non_specified_features=True, 
    random_seed=1234
)
model.fit(train_ds)
self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} | Loss:{self_evaluation.loss}")

[WARNING 24-01-12 15:38:54.7892 +07 gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:38:54.7896 +07 gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:38:54.7897 +07 gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
2024-01-12 15:38:56.795688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
[INFO 24-01-12 15:38:57.0047 +07 kernel.cc:1233] Loading model from path /var/folders/08/pfjkw6qj6432p465r25wkxf80000gp/T/tmplw89s38y/model/ with prefix 2afc2238b1b8443c
[INFO 24-01-12 15:38:57.0078 +07 abstract_model.cc:1344] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 24-01-12 15:38:57.0078 +07 kernel.cc:1061] Use fast generic engine


Accuracy: 0.804347813129425 | Loss:0.8854019641876221


In [7]:
model.summary()

Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.391099 ################
    2.          "Name"  0.288523 ########
    3.           "Age"  0.238785 ####
    4.        "Pclass"  0.233203 ####
    5.          "Fare"  0.230704 ###
    6. "Ticket_number"  0.212794 ##
    7.   "Ticket_item"  0.199716 #
    8.      "Embarked"  0.192019 
    9.         "SibSp"  0.182693 
   10.         "Parch"  0.180443 

Variable Importance: NUM_AS_ROOT:
    1.           "Sex

In [9]:
tuner = tfdf.tuner.RandomSearch(num_trials=1000)
tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

#tuner.choice("use_hessian_gain", [True, False])
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])


tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization",
                     ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])

# Tune the model. Notice the `tuner=tuner`.
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=0)

tuned_self_evaluation = tuned_model.make_inspector().evaluation()
print(f"Accuracy: {tuned_self_evaluation.accuracy} Loss:{tuned_self_evaluation.loss}")

Use /var/folders/08/pfjkw6qj6432p465r25wkxf80000gp/T/tmptll3p3pa as temporary training directory


[WARNING 24-01-12 15:39:22.8682 +07 gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:39:22.8682 +07 gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:39:22.8682 +07 gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Accuracy: 0.8767123222351074 Loss:0.6447146534919739


[INFO 24-01-12 15:39:42.8763 +07 kernel.cc:1233] Loading model from path /var/folders/08/pfjkw6qj6432p465r25wkxf80000gp/T/tmptll3p3pa/model/ with prefix c58dd7b409424a16
[INFO 24-01-12 15:39:42.8833 +07 decision_forest.cc:660] Model loaded with 30 root(s), 1760 node(s), and 12 input feature(s).
[INFO 24-01-12 15:39:42.8833 +07 abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-01-12 15:39:42.8833 +07 kernel.cc:1061] Use fast generic engine


## Ensemble
Here you'll create 100 models with different seeds and combine their results

This approach removes a little bit the random aspects related to creating ML models

In the GBT creation is used the `honest` parameter. It will use different training examples to infer the structure and the leaf values. This regularization technique trades examples for bias estimates.

In [19]:
predictions = None
num_predictions = 0
for i in range(100): 
    print('i:', i)
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0, 
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features], 
        exclude_non_specified_features=True, 
        random_seed=i, 
        honest=True
    )
    model.fit(train_ds)
    
    sub_predictions = model.predict(serving_ds, verbose=0)[:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1

i: 0


[WARNING 24-01-12 15:50:15.1451 +07 gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:50:15.1451 +07 gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:50:15.1451 +07 gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-01-12 15:50:15.4435 +07 kernel.cc:1233] Loading model from path /var/folders/08/pfjkw6qj6432p465r25wkxf80000gp/T/tmpatjw48xo/model/ with prefix e550a1a7da3646f1
[INFO 24-01-12 15:50:15.4455 +07 abstract_model.cc:1344] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 24-01-12 15:50:15.4455 +07 kernel.cc:1061] Use fast generic engine


i: 1


[WARNING 24-01-12 15:50:15.6514 +07 gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:50:15.6514 +07 gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:50:15.6514 +07 gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-01-12 15:50:15.9970 +07 kernel.cc:1233] Loading model from path /var/folders/08/pfjkw6qj6432p465r25wkxf80000gp/T/tmpkaxajdk2/model/ with prefix 06b9a9ae7314480c
[INFO 24-01-12 15:50:16.0022 +07 kernel.cc:1061] Use fast generic engine


i: 2


...

[WARNING 24-01-12 15:51:07.6486 +07 gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:51:07.6487 +07 gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:51:07.6487 +07 gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-01-12 15:51:07.9422 +07 kernel.cc:1233] Loading model from path /var/folders/08/pfjkw6qj6432p465r25wkxf80000gp/T/tmpr5yc0xfk/model/ with prefix 41354a642d1f4141
[INFO 24-01-12 15:51:07.9452 +07 kernel.cc:1061] Use fast generic engine


i: 98


[WARNING 24-01-12 15:51:08.1540 +07 gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:51:08.1540 +07 gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:51:08.1540 +07 gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-01-12 15:51:08.5057 +07 kernel.cc:1233] Loading model from path /var/folders/08/pfjkw6qj6432p465r25wkxf80000gp/T/tmp8rbw8eo6/model/ with prefix f0090cef5042450d
[INFO 24-01-12 15:51:08.5104 +07 kernel.cc:1061] Use fast generic engine


i: 99


[WARNING 24-01-12 15:51:08.7184 +07 gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:51:08.7184 +07 gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-12 15:51:08.7184 +07 gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-01-12 15:51:08.9870 +07 kernel.cc:1233] Loading model from path /var/folders/08/pfjkw6qj6432p465r25wkxf80000gp/T/tmppm5cp__s/model/ with prefix 0a99a7a8dd864546
[INFO 24-01-12 15:51:08.9892 +07 kernel.cc:1061] Use fast generic engine


In [22]:
predictions/=num_predictions
kaggle_predictions = pd.DataFrame({
    "PassengerId": serving_df['PassengerId'], 
    "Survived": (predictions >= 0.5).astype(int)
})  
kaggle_predictions.to_csv('my_tf_tunning.csv', index=False)